<a href="https://colab.research.google.com/github/Rafaabreu1/Projeto-An-lise-de-Dados-da-COVID-19-Dashboard-2022/blob/main/Projeto_An%C3%A1lise_de_Dados_da_COVID_19_Dashboard_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **COVID-19 Dashboard**

## 1\. Introdução

### **1.1. TLDR**

 - **Dashboard**:
  - Google Data Studio ([link](https://lookerstudio.google.com/reporting/33ff63e8-4724-48df-b93b-59c6dbbc824a)).
 - **Processamento**:
  - Kaggle Notebook ([link](https://www.kaggle.com/code/rafaeldeabreu/notebook19411c4008/edit)).
 - **Fontes**:
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

 1.2. Contexto

A disponibilidade de dados sobre a evolução da pandemia no tempo em uma determinada região geográfica é fundamental para o seu combate! Este projeto busca construir um dashboard de dados para exploração e visualização interativa de dados sobre o avanço de casos e da vacinação do Brasil. O processamento de dados está neste `link` e o dashboard, neste ([link](https://lookerstudio.google.com/reporting/33ff63e8-4724-48df-b93b-59c6dbbc824a)).

## 2\. Pacotes e bibliotecas

In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## 3\. Extração

In [2]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/refs/heads/master/csse_covid_19_data/csse_covid_19_daily_reports/01-01-2022.csv', sep=',')

In [3]:
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-01-02 04:20:52,33.93911,67.709953,158107,7356,NaN,NaN,Afghanistan,406.148826,4.652545
1,NaN,NaN,NaN,Albania,2022-01-02 04:20:52,41.15330,20.168300,210224,3217,NaN,NaN,Albania,7305.024672,1.530272
2,NaN,NaN,NaN,Algeria,2022-01-02 04:20:52,28.03390,1.659600,218818,6284,NaN,NaN,Algeria,499.002954,2.871793
3,NaN,NaN,NaN,Andorra,2022-01-02 04:20:52,42.50630,1.521800,23740,140,NaN,NaN,Andorra,30725.425484,0.589722
4,NaN,NaN,NaN,Angola,2022-01-02 04:20:52,-11.20270,17.873900,82398,1772,NaN,NaN,Angola,250.706895,2.150538


In [4]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [5]:
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)

In [6]:
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/refs/heads/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))
  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], axis= 0, ignore_index=True)

In [7]:
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,88386,1851,10021.827005,2022-01-01
1,Alagoas,Brazil,242091,6383,7253.973728,2022-01-01
2,Amapa,Brazil,127013,2022,15018.132243,2022-01-01
3,Amazonas,Brazil,433850,13836,10467.845245,2022-01-01
4,Bahia,Brazil,1271029,27507,8545.845026,2022-01-01


## 4\. Transformação

In [8]:
cases= cases.rename(columns={'Province_State': 'state', 'Country_Region': 'country'})
cases.columns = cases.columns.str.lower()

In [9]:
cases.head()

,state,country,confirmed,deaths,incident_rate,date
0,Acre,Brazil,88386,1851,10021.827005,2022-01-01
1,Alagoas,Brazil,242091,6383,7253.973728,2022-01-01
2,Amapa,Brazil,127013,2022,15018.132243,2022-01-01
3,Amazonas,Brazil,433850,13836,10467.845245,2022-01-01
4,Bahia,Brazil,1271029,27507,8545.845026,2022-01-01


In [10]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

In [11]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

In [12]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.85:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status


for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_, cases_per_state],axis=0, ignore_index=True)

cases = cases_
cases_ = None

In [13]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [14]:
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [15]:
cases.head()

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2022-01-01,Brazil,Acre,881935,88386,<NA>,<NA>,NaN,NaN,1851,<NA>,<NA>,NaN,NaN,2022-01,2022
1,2022-01-02,Brazil,Acre,881935,88386,0,<NA>,NaN,NaN,1851,0,<NA>,NaN,NaN,2022-01,2022
2,2022-01-03,Brazil,Acre,881935,88395,9,<NA>,NaN,NaN,1851,0,<NA>,NaN,NaN,2022-01,2022
3,2022-01-04,Brazil,Acre,881935,88395,0,<NA>,NaN,NaN,1852,1,<NA>,NaN,NaN,2022-01,2022
4,2022-01-05,Brazil,Acre,881935,88395,0,<NA>,NaN,NaN,1852,0,<NA>,NaN,NaN,2022-01,2022


## 5\. Carregamento

In [16]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

#6\. Vacinação

##6.1\.Extração

In [17]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

<ipython-input-17-608e6d2dd56d>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


In [18]:
vaccines.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN


In [19]:
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

## **6.2. Wrangling**

In [20]:
vaccines = vaccines.fillna(method='ffill')

<ipython-input-20-274988f553eb>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vaccines = vaccines.fillna(method='ffill')


In [21]:
vaccines = vaccines[(vaccines['date'] >= '2022-01-01') & (vaccines['date'] <= '2022-12-31')].reset_index(drop=True)

In [22]:
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

In [23]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

In [24]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

In [25]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

In [26]:
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

In [27]:
vaccines.tail()

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
360,2022-12-27,Brazil,215313504,480310839,188549744,0.8757,174881292,0.8122,122616211,0.5695,2022-12,2022
361,2022-12-28,Brazil,215313504,480310839,188549744,0.8757,174881292,0.8122,122616211,0.5695,2022-12,2022
362,2022-12-29,Brazil,215313504,480331769,188552661,0.8757,174886102,0.8122,122629436,0.5695,2022-12,2022
363,2022-12-30,Brazil,215313504,480332769,188553047,0.8757,174886846,0.8122,122629436,0.5695,2022-12,2022
364,2022-12-31,Brazil,215313504,480333910,188553932,0.8757,174887915,0.8122,122629436,0.5695,2022-12,2022


## **6.3. Carregamento**

In [28]:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

## 7\. Exploração Interativa de Dados



### **7.1. KPIs**

O dashboard de dados contem os seguintes indicadores chaves de desempenho (*key performance indicator* ou KPI) consolidados:

1. Casos e mortes nas 24 horas;
1. Média móvel (7 dias) de casos e mortes;
1. Tendência de casos e mortes;
1. Proporção de vacinados com 1ª, 2ª e 3ª doses.

### **7.2. EDA**

O dashboard de dados contem os seguintes gráficos para a análise exploratória de dados (*exploratory data analysis*
ou EDA) interativa:

1. Distribuição do números de casos e mortes ao longo do tempo;
1. Distribuição da média móvel (7 dias) do números de casos e mortes ao longo do tempo;
1. Distribuição geográfica dos casos por estado por dia.

Dashboard Covid-19: ([link](https://lookerstudio.google.com/reporting/33ff63e8-4724-48df-b93b-59c6dbbc824a)).

## 8\. Storytelling



O ano de 2022 marcou um período crucial no enfrentamento da COVID-19 no Brasil, com a vacinação desempenhando um papel essencial na mitigação dos impactos da doença. Neste texto, serão apresentados dados sobre a adesão às doses de vacina, além da evolução temporal dos casos confirmados e de mortes. Essas análises oferecem um panorama sobre os avanços alcançados e os desafios ainda enfrentados.

## Vacinação no Brasil em 2022

No ano de 2022, o Brasil registrou a aplicação de 480.332.769 doses de vacinas, abrangendo as três principais etapas de imunização:

1ª dose: Cobertura de 87,6% da população.

2ª dose: Cobertura de 81,2% da população.

3ª dose: Apenas 57% da população aderiu à dose de reforço.

Esses dados mostram que a maioria da população completou o esquema vacinal básico (1ª e 2ª doses). No entanto, a adesão à dose de reforço foi significativamente menor, o que representa um risco, especialmente em um cenário de variantes mais transmissíveis. Isso destaca a importância de campanhas educativas que reforcem a necessidade de completar todas as etapas da imunização, reduzindo assim o risco de complicações graves e mortes.

## Evolução Temporal dos Casos de Mortes

A análise da evolução temporal das mortes por COVID-19 em 2022, baseada em médias móveis de 7 dias e 24 horas, revela tendências importantes:

## Meses iniciais:

 Alta incidência de óbitos, com pico no mês de fevereiro.

## Redução em abril:

 Após o pico, observou-se uma queda significativa no número de mortes.
Aumento em julho e agosto:

 Um leve aumento foi registrado nesses meses.

## Redução contínua:

 Após agosto, os números voltaram a cair gradualmente.
Esses dados indicam que a sazonalidade e fatores como adesão à vacinação e medidas de prevenção influenciaram diretamente a mortalidade ao longo do ano.

## Evolução Temporal dos Casos Confirmados

O comportamento dos casos confirmados de COVID-19 ao longo de 2022 apresentou uma trajetória semelhante à das mortes:

## Pico em fevereiro:

 A maior incidência de casos ocorreu nos primeiros meses do ano.
Queda em março: Seguiu-se uma redução significativa no número de casos.
Novo aumento em abril e maio: Esses meses apresentaram um leve crescimento no número de infectados.

## Redução constante:

 Nos meses seguintes, os números continuaram em queda.
Os gráficos evidenciam a relação entre o número de casos e as mortes, ressaltando a importância de medidas de prevenção para conter o avanço da doença.

Os dados analisados mostram que, embora o Brasil tenha avançado significativamente na vacinação em 2022, há desafios a serem enfrentados:

### Adesão à dose de reforço:

 Campanhas mais eficazes são necessárias para aumentar a adesão à 3ª dose, garantindo maior proteção à população.

### Monitoramento contínuo:

 A vigilância constante é essencial para identificar picos de transmissão e implementar ações rápidas.

### Educação e conscientização:

Informar a população sobre a importância da vacinação e do uso de medidas preventivas é crucial para reduzir o impacto da COVID-19.
Com estratégias alinhadas e baseadas em dados, é possível garantir avanços no combate à pandemia, proteger vidas e preparar o país para enfrentar desafios futuros.
